In [1]:
import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.rdd.RDD
import scala.collection.mutable.ListBuffer
import scala.io.Source

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.3:4040
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1605546657647)
SparkSession available as 'spark'


import org.apache.spark.SparkConf
import org.apache.spark.SparkContext
import org.apache.spark.rdd.RDD
import scala.collection.mutable.ListBuffer


## Task 1

In [38]:
import scala.io.Source
implicit def bool2int(b:Boolean) = if (b) 1 else 0

def apriori_alg(data: String, support:Int, k:Int): List[List[Int]] = {
    
    """
    A-Priori algorithm for finding frequent itemsets with support at least 'support' in data.
    
    Params:
    - data: path to the dataset
    - support: support threshold
    - k: size of the itemset
    
    Returns: 
    - List with the frequent itemsets (by calling the function count_itemsets).
    """
    
    val source = Source.fromFile(data)
    val singletons = Array.fill(100000)(0)
    val candidates = Array.range(0, 1000) 

    for (line <- source.getLines().map(x => x.split(" ").map(item => item.toInt))) {
        for (i <- candidates){
            singletons(i) = singletons(i) + List(i).forall(line.toList.contains)
            }
        }
    source.close()

    val frequent_singl = singletons.zip(candidates).filter(x => (x._1 >= support)).map(item => item._2)
    val candidates2 = frequent_singl.toSet[Int].subsets(2).map(_.toList).toList
    
    return count_itemsets(data, support, 2, candidates2)
}
 


def count_itemsets(data: String, support:Int, k:Int, candidates: List[List[Int]]): List[List[Int]] = {
    """
    Recursive function which searchs for frequent itemsets with k>1.
    
    Params:
    - data: path to dataset
    - support: support threshold
    - k: size of the itemset
    - candidates: possible frequent itemset
    
    Returns:
    - frequent_itemsets
    """
    
    var i = 0
    val occurrences = Array.fill(candidates.size)(0)
    val frequent_itemsets = new ListBuffer[(List[List[Int]])]()
    
    println("open source",k)
    val source = Source.fromFile(data)
    for (line <- source.getLines().map(x => x.split(" ").map(item => item.toInt))) {
        if (i%1000 == 0){
        println("lines done ",i)
        }
        for (item <- candidates.zipWithIndex) {
            occurrences(item._2) = occurrences(item._2)  + item._1.forall(line.toList.contains)
        }
        i = i + 1
    }
    source.close()
    println("close source",k)
    
    val frequent_sets_k = occurrences.zip(candidates).filter(x => (x._1 >= support)).map(item => item._2).toList
    frequent_itemsets += frequent_sets_k
    
    if (frequent_sets_k.size >1) {
        val candidates2 = frequent_sets_k.flatten.toSet[Int].subsets(k+1).map(_.toList).toList
        
        frequent_itemsets += count_itemsets(data, support, k+1, candidates2)
    }
    
    return frequent_itemsets.toList.flatten
} 


import scala.io.Source
bool2int: (b: Boolean)Int
apriori_alg: (data: String, support: Int, k: Int)List[List[Int]]
count_itemsets: (data: String, support: Int, k: Int, candidates: List[List[Int]])List[List[Int]]


In [22]:
// parameters 
val data = "data/T10I4D100K.dat"
val support = 1000
val k = 1

data: String = data/T10I4D100K.dat
support: Int = 1000


In [40]:
val frequent_itmsets = apriori_alg(data, support, k)

(open source,2)
(lines done ,0)
(lines done ,1000)
(lines done ,2000)
(lines done ,3000)
(lines done ,4000)
(lines done ,5000)
(lines done ,6000)
(lines done ,7000)
(lines done ,8000)
(lines done ,9000)
(lines done ,10000)
(lines done ,11000)
(lines done ,12000)
(lines done ,13000)
(lines done ,14000)
(lines done ,15000)
(lines done ,16000)
(lines done ,17000)
(lines done ,18000)
(lines done ,19000)
(lines done ,20000)
(lines done ,21000)
(lines done ,22000)
(lines done ,23000)
(lines done ,24000)
(lines done ,25000)
(lines done ,26000)
(lines done ,27000)
(lines done ,28000)
(lines done ,29000)
(lines done ,30000)
(lines done ,31000)
(lines done ,32000)
(lines done ,33000)
(lines done ,34000)
(lines done ,35000)
(lines done ,36000)
(lines done ,37000)
(lines done ,38000)
(lines done ,39000)
(lines done ,40000)
(lines done ,41000)
(lines done ,42000)
(lines done ,43000)
(lines done ,44000)
(lines done ,45000)
(lines done ,46000)
(lines done ,47000)
(lines done ,48000)
(lines done ,4900

frequent_itmsets: List[List[Int]] = List(List(217, 346), List(829, 789), List(829, 368), List(825, 704), List(825, 39), List(682, 368), List(704, 39), List(722, 390), List(390, 227), List(825, 704, 39))


## Task 2

In [41]:
val items = frequent_itmsets.flatten.map(x => List(x)) ::: frequent_itmsets
val occurrences = Array.fill(items.size)(0)
val source = Source.fromFile(data)

for (line <- source.getLines().map(x => x.split(" ").map(item => item.toInt))) {
    for (item <- items.zipWithIndex) {
                occurrences(item._2) = occurrences(item._2)  + item._1.forall(line.toList.contains)
    }
}
source.close()
println("\nDONE")


DONE


items: List[List[Int]] = List(List(217), List(346), List(829), List(789), List(829), List(368), List(825), List(704), List(825), List(39), List(682), List(368), List(704), List(39), List(722), List(390), List(390), List(227), List(825), List(704), List(39), List(217, 346), List(829, 789), List(829, 368), List(825, 704), List(825, 39), List(682, 368), List(704, 39), List(722, 390), List(390, 227), List(825, 704, 39))
occurrences: Array[Int] = Array(5375, 3470, 6810, 4309, 6810, 7828, 3085, 1794, 3085, 4258, 4132, 7828, 1794, 4258, 5845, 2685, 2685, 1818, 3085, 1794, 4258, 1336, 1194, 1194, 1102, 1187, 1193, 1107, 1042, 1049, 1035)
source: scala.io.BufferedSource = <iterator>


In [58]:
def association_rule(itemset: List[Int], support_set: List[(List[Int], Int)], thr: Double): Int = {
    """
    Algorithm for generating association rules between frequent itemsets discovered by using the 
    A-Priori algorithm in a dataset of sales transactions.
    
    Params:
    - itemset: single frequent itemset
    - support_set: contains the support of each element of the itemsets
    - th: threshold of confidence
    
    Returns:
    - Prints the association rule with confidence > threshold.
    """
    
    //println("Itemset: $itemset")
    val left_rule = itemset.toSet[Int].subsets().map(_.toList).toList.drop(1).dropRight(1)
    val num = support_set.filter(x => x._1 == itemset).map(x => x._2: Double)
    for (i <- left_rule){
        val den = support_set.filter(x => x._1 == i).map(x => x._2: Double)
        val confidence: Double = num(0)/den(0)
        val right_rule = itemset diff i
        if (confidence>= thr) {
            println(f"{$i} -> {$right_rule}: $confidence")
        }
    }
    return 0
}

<console>:66: warning: a pure expression does nothing in statement position; multiline expressions might require enclosing parentheses
           """
           ^
association_rule: (itemset: List[Int], support_set: List[(List[Int], Int)], thr: Double)Int


In [59]:
val c = 0.5 // confidence thr
frequent_itmsets.foreach{association_rule(_, items.zip(occurrences), c)}

{List(704)} -> {List(825)}: 0.6142697881828316
{List(704)} -> {List(39)}: 0.617056856187291
{List(227)} -> {List(390)}: 0.577007700770077
{List(704)} -> {List(825, 39)}: 0.5769230769230769
{List(825, 704)} -> {List(39)}: 0.9392014519056261
{List(825, 39)} -> {List(704)}: 0.8719460825610783
{List(704, 39)} -> {List(825)}: 0.9349593495934959


c: Double = 0.5
